In [8]:
import pandas as pd
import numpy as np
import pandasql as ps

from pandasql import sqldf

In [9]:
course_df = pd.read_csv("../../data/scheduling/input/raw/Sample data (bestco version) - course_name.csv")
student_df = pd.read_csv("../../data/scheduling/input/raw/Sample data (bestco version) - student_seasonal_course.csv")
teacher_df = pd.read_csv("../../data/scheduling/input/raw/Sample data (bestco version) - teacher_seasonal_course.csv")

# Student

In [10]:
std_course_df = student_df.merge(course_df, how="left",
                                 left_on = ["regular_course"],
                                 right_on = ["course_name"]) 
std_course_df = std_course_df.merge(course_df, how="left",
                                 left_on = ["seasonal_course"],
                                 right_on = ["course_name"]) 

In [11]:
def filter_time(row_time):
    # have 6 time shift per day
    avail_time = []
    unavail_time = row_time.split(",")
    for i in range (6):
        if not (str(i) in unavail_time):
            avail_time.append(i)
        
    return avail_time

std_course_df["available_time"] = std_course_df["unavailable_time"].apply(lambda row: filter_time(row))

In [12]:
def filter_day(row_day):
    # have time range 60 days
    avail_day = []
    unavail_day = row_day.split(",")
    for i in range (14):
        if not (str(i) in unavail_day):
            avail_day.append(i)
        
    return avail_day

std_course_df["available_day"] = std_course_df["unavailable_day"].apply(lambda row: filter_day(row))

In [13]:
def get_subject(row, is_season=False):
    subject_list = []
    if is_season:
        if row["math_y"] > 0:
            subject_list.append("math")
        if row["english_y"] > 0:
            subject_list.append("english")
        if row["literature_y"] > 0:
            subject_list.append("literature")
        if row["social_science_y"] > 0:
            subject_list.append("social_science")
        if row["science_y"] > 0:
            subject_list.append("science")
    else:
        if row["math_x"] > 0:
            subject_list.append("math")
        if row["english_x"] > 0:
            subject_list.append("english")
        if row["literature_x"] > 0:
            subject_list.append("literature")
        if row["social_science_x"] > 0:
            subject_list.append("social_science")
        if row["science_x"] > 0:
            subject_list.append("science")
    return subject_list
                           

In [14]:
std_course_df["regular_subject"] = std_course_df.apply(lambda row: get_subject(row, is_season=False), axis=1)
std_course_df["season_subject"] = std_course_df.apply(lambda row: get_subject(row, is_season=True), axis=1)
std_course_df["num_day_avail"] = std_course_df[["student","available_time", "available_day"]].apply(lambda row: len(row["available_day"]), axis=1) 
std_course_df["subject"] = std_course_df.apply(lambda row: list(set(row["regular_subject"] + row["season_subject"])), axis=1)

std_course_df_formated = std_course_df.explode("available_day").explode("available_time").explode("regular_subject").explode("subject")

# std_course_df_formated.to_csv("./formated/student_course_formated.csv")

In [15]:

std_course_df[["student","available_time", "available_day", "num_day_avail"]].sort_values(by="num_day_avail")

,student,available_time,available_day,num_day_avail
25,st_26,"[0, 1, 5]","[0, 1, 2, 3, 8, 9, 10, 11, 12]",9
23,st_24,"[0, 3, 4]","[0, 1, 2, 3, 8, 9, 10, 11, 12]",9
2,st_3,"[0, 1, 2]","[1, 3, 4, 5, 7, 8, 9, 10, 12]",9
3,st_4,"[0, 3, 4]","[1, 2, 3, 4, 5, 8, 9, 10, 11]",9
21,st_22,"[0, 1, 5]","[0, 1, 2, 3, 4, 7, 9, 10, 11]",9
8,st_9,"[0, 4, 5]","[1, 2, 3, 4, 5, 8, 10, 11, 12]",9
9,st_10,"[0, 1, 5]","[0, 2, 3, 4, 8, 9, 10, 11, 12]",9
16,st_17,"[0, 4, 5]","[0, 3, 4, 5, 8, 9, 10, 11, 12]",9
13,st_14,"[0, 1, 5]","[0, 1, 3, 4, 5, 7, 9, 10, 11]",9
14,st_15,"[0, 1, 2]","[0, 2, 4, 5, 7, 9, 10, 11, 12]",9


In [16]:
# std_course_df.to_csv("./formated/std_course_df.csv")

# Teacher

In [17]:
teacher_df

,teacher,subject,unavailable_time,unavailable_day,priority,location,grade
0,t_1,"math,science","1,2,3,4","9,0,1,6,13",1,a,"g1,g2,g3"
1,t_2,"english,social_science","3,4","6,10,13,0",2,b,"g4,g5,g6"
2,t_3,"literature,social_science","2,3,4","8,6,5,3,13",3,c,"g7,g8,g9"
3,t_4,english,"4,5","6,13,2,5",4,"a,b,c","g10,g11"
4,t_5,math,"1,3,5","6,9,13,13",5,"b,c",g12
5,t_6,"science,social_science","2,4,5","9,0,6,5,13",6,"a,c",g12
6,t_7,"english,literature","1,2,3,4","0,8,6,13,2",7,"a,b,c","g1,g2,g3"
7,t_8,"math,science","4,5","11,13,6,13,2",8,"b,c","g4,g5,g6"
8,t_9,"english,social_science","2,3,4","9,0,6,13,2",9,"a,c","g7,g8,g9"
9,t_10,"literature,social_science","4,5","6,13,0",10,a,"g10,g11"


In [18]:
teacher_df["available_time"] = teacher_df["unavailable_time"].apply(lambda row: filter_time(row))
teacher_df["available_day"] = teacher_df["unavailable_day"].apply(lambda row: filter_day(row))
teacher_df["subject"] = teacher_df["subject"].apply(lambda row: row.split(","))
teacher_df["grade"] = teacher_df["grade"].apply(lambda row: row.split(","))
teacher_df["location"] = teacher_df["location"].apply(lambda row: row.split(","))
teacher_df

,teacher,subject,unavailable_time,unavailable_day,priority,location,grade,available_time,available_day
0,t_1,"[math, science]","1,2,3,4","9,0,1,6,13",1,[a],"[g1, g2, g3]","[0, 5]","[2, 3, 4, 5, 7, 8, 10, 11, 12]"
1,t_2,"[english, social_science]","3,4","6,10,13,0",2,[b],"[g4, g5, g6]","[0, 1, 2, 5]","[1, 2, 3, 4, 5, 7, 8, 9, 11, 12]"
2,t_3,"[literature, social_science]","2,3,4","8,6,5,3,13",3,[c],"[g7, g8, g9]","[0, 1, 5]","[0, 1, 2, 4, 7, 9, 10, 11, 12]"
3,t_4,[english],"4,5","6,13,2,5",4,"[a, b, c]","[g10, g11]","[0, 1, 2, 3]","[0, 1, 3, 4, 7, 8, 9, 10, 11, 12]"
4,t_5,[math],"1,3,5","6,9,13,13",5,"[b, c]",[g12],"[0, 2, 4]","[0, 1, 2, 3, 4, 5, 7, 8, 10, 11, 12]"
5,t_6,"[science, social_science]","2,4,5","9,0,6,5,13",6,"[a, c]",[g12],"[0, 1, 3]","[1, 2, 3, 4, 7, 8, 10, 11, 12]"
6,t_7,"[english, literature]","1,2,3,4","0,8,6,13,2",7,"[a, b, c]","[g1, g2, g3]","[0, 5]","[1, 3, 4, 5, 7, 9, 10, 11, 12]"
7,t_8,"[math, science]","4,5","11,13,6,13,2",8,"[b, c]","[g4, g5, g6]","[0, 1, 2, 3]","[0, 1, 3, 4, 5, 7, 8, 9, 10, 12]"
8,t_9,"[english, social_science]","2,3,4","9,0,6,13,2",9,"[a, c]","[g7, g8, g9]","[0, 1, 5]","[1, 3, 4, 5, 7, 8, 10, 11, 12]"
9,t_10,"[literature, social_science]","4,5","6,13,0",10,[a],"[g10, g11]","[0, 1, 2, 3]","[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12]"


In [19]:

teacher_df_formated = teacher_df.explode("subject").explode("grade").explode("available_time").explode("available_day").explode("location")
# teacher_df_formated.to_csv("./formated/teacher_formated.csv")

In [20]:
teacher_df_formated

,teacher,subject,unavailable_time,unavailable_day,priority,location,grade,available_time,available_day
0,t_1,math,"1,2,3,4","9,0,1,6,13",1,a,g1,0,2
0,t_1,math,"1,2,3,4","9,0,1,6,13",1,a,g1,0,3
0,t_1,math,"1,2,3,4","9,0,1,6,13",1,a,g1,0,4
0,t_1,math,"1,2,3,4","9,0,1,6,13",1,a,g1,0,5
0,t_1,math,"1,2,3,4","9,0,1,6,13",1,a,g1,0,7
...,...,...,...,...,...,...,...,...,...
14,t_15,social_science,"1,2,3,4","6,13,2,3",15,a,g12,5,8
14,t_15,social_science,"1,2,3,4","6,13,2,3",15,a,g12,5,9
14,t_15,social_science,"1,2,3,4","6,13,2,3",15,a,g12,5,10
14,t_15,social_science,"1,2,3,4","6,13,2,3",15,a,g12,5,11
